### ライブラリのインポート

In [8]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os
import math
import optuna
import re
import random
import warnings

# 全ての警告を無視する
warnings.filterwarnings("ignore")

# OptunaのロギングレベルをERRORに設定
optuna.logging.set_verbosity(optuna.logging.ERROR)

### 計算用関数

In [9]:
# 行数を指定すると所用時間を返す関数
def knapsack(row_x, data_x,required_time_x, waiting_time_x, move_time_x, attraction_no_x):
    waiting_time_x = data_x.iloc[row_x, attraction_no_x]
    ans = required_time_x + waiting_time_x + move_time_x
    return ans


# 合計所要時間と合計人気度を算出する関数
def ride_calculation(data, ride_lst, required_times, popularity, move_time):
    # print(data)
    # print(ride_lst)
    # print(required_times)
    # print(move_time)
    total_popularity = 0
    total_time = 0
    ride_time = []
    new_ride_lst = []
    # start_time = data.index[0]
    # print("start time is:", calculate_time(start_time))
    row = 0
    try:
        c=0
        for i in ride_lst:
            # total_timeが675以上かつiが0または1の場合にループを強制終了
            # ソアリン、トイストーリーMは20:15以降DPA所持者のみ乗車可能
            if (data.index[row]) >= 675 and (i == 0 or i == 1):
                c+=1
                continue
            
            else:
                # iが8以上の場合、次のループへ(該当しないアトラクション)
                if i >= 8 or i < 0:
                    # print("不明なアトラクション：", i)
                    c+=1
                    continue
                elif c>0 and i==ride_lst[c-1]:
                    c+=1
                    continue
                else:
                    # print("-" * 40)
                    # print(calculate_time(data.index[row]))
                    ride_time.append(data.index[row])
                    # print(attractions.get(i))
                    # print("移動時間", move_time)
                    waiting_time = data.iloc[row, i]
                    # print("待ち時間:", waiting_time)
                    required_time = required_times[i]
                    # print("観賞時間:", required_time)
                    ans = knapsack(row, data, required_time, waiting_time, move_time, i)
                    # print("合計所要時間:", ans)
                    total_time += ans
                    row = int(np.ceil(total_time / 15))
                    total_popularity += popularity[i]
                    new_ride_lst.append(i)
                    c+=1
    except Exception as e:
        # print("全てのアトラクションを乗ることができません")
        pass

    # print("-" * 40)
    # print(total_popularity)
    return total_popularity, total_time, new_ride_lst, ride_time

In [10]:
df_j_opt=pd.DataFrame(columns=['date', 'lamb','value','total_time','scheduled_plan','optimal_plan','time_schedule'])
display(df_j_opt.head())

,date,lamb,value,total_time,scheduled_plan,optimal_plan,time_schedule


### 最適化実行関数

In [11]:
def optimization(file_pathes):
    best_plan_lst = []
    file_path = file_pathes
    
    lamb_list=np.linspace(1,20,10)
    #print(lamb_list)
    tt_best=720
    j=0
    NN=300
    while(j<10):
    #while(np.abs(tt_best-615)>60):
    #for j in range(2):
        #print(tt_best)
        total_time_list=[]
        

        # 目的関数の定義
        def objective(trial):
            # アトラクションの所要時間と人気度 or レビュー評価
            required_times = [5, 7, 2, 3, 3, 2, 23, 30]  # 所要時間
            popularity = [476, 465, 473, 472, 480, 452, 461, 478] # レビュー評価
            #popularity = [84, 69, 55, 45, 33, 28, 19, 22]  # 人気度
            move_time = 15

            # print(file_path)
            data = pd.read_csv(file_path)
            data = data.set_index('時間')

            # 乗り物の数を決定
            n_rides = trial.suggest_int('n_rides', 10, 20)  # 最大20の乗り物を仮定

            # 乗り物の定義
            ride_lst = [trial.suggest_int(f'ride_{i}', 0, 7) for i in range(n_rides)]

            # ride_calculation関数の呼び出し
            total_popularity, total_time, new_ride_lst, ride_time = ride_calculation(data, ride_lst, required_times, popularity, move_time)

            # 制約条件の範囲
            #max_allowed_time = 660  # パーク内にいる時間 9:00~20:00 計660分 (20時以降はファストパスのみ受け入れと仮定)
            max_allowed_time = 675  # パーク内にいる時間 9:00~20:15 計675分 (20:15以降はファストパスのみ受け入れと仮定)
            
            #trial.set_user_attr("constraints", [1000*(max_allowed_time - total_time)])
            #print("total_time:",total_time)
            #print("total_popularity:",total_popularity)
            total_time_list.append(total_time)
            
            # 正規表現で年月日部分を抽出
            match = re.search(r'\d{4}-\d{2}-\d{2}', file_path)
            date = match.group() if match else None
            best_plan_lst.append([trial.number, date, total_popularity, total_time, ride_lst, new_ride_lst, ride_time])
            lamb=lamb_list[j]
            #print('lambda is',lamb)
            
            return total_popularity+lamb*(max_allowed_time - total_time)
            

            # 制約を満たす場合のみ最適化対象とする
            #if total_time <= max_allowed_time:
                # 正規表現で年月日部分を抽出
                #match = re.search(r'\d{4}-\d{2}-\d{2}', file_path)
                #date = match.group() if match else None

                #best_plan_lst.append([trial.number, date, total_popularity, total_time, ride_lst, new_ride_lst, ride_time])
                # print(total_popularity)
                #return total_popularity
            #else:
                # 制約を満たさない場合はペナルティを課す
                #return total_popularity+(max_allowed_time-total_time)*100


            #####
            # 制約を満たす場合のみ最適化対象とする
            #trial.set_user_attr("constraints",[max_allowed_time - total_time])
            # 正規表現で年月日部分を抽出
            #match = re.search(r'\d{4}-\d{2}-\d{2}', file_path)
            #date = match.group() if match else None
            #best_plan_lst.append([trial.number, date, total_popularity, total_time, ride_lst, new_ride_lst, ride_time])
            # print(total_popularity)
            #return total_popularity
            #####



        # Optunaによる最適化
        study = optuna.create_study(direction="maximize")  # 人気度を最大化
        study.optimize(objective, n_trials = NN)
        best_trial = study.best_trial
        tt_best = total_time_list[best_trial.number] #最適解における所要時間の値。

        best_trial = study.best_trial
        #print(f"Best trial: {best_trial.number}")
        #print(f"Best popularity: {best_trial.value}")
        #print(f"Parameters: {best_trial.params}")
        #print("total_time here is:", tt_best)

        j=j+1
        #print("tt_best2:",tt_best)

            
            
        # DataFrameに変換
        best_plan_df = pd.DataFrame(best_plan_lst)
        #print("best_plan_df:")
        #display(best_plan_df)

        # best_trial.numberに対応する行を抽出
        best_plan = best_plan_df[best_plan_df[0] == best_trial.number].iloc[0]
        #print("best_plan:")
        #display(best_plan)
        df_j_opt.loc[j,'date'] =  best_plan[1]        
        df_j_opt.loc[j,'lamb'] = j
        df_j_opt.loc[j,'value'] = best_plan[2]
        df_j_opt['value'] = df_j_opt['value'].astype(float)
        df_j_opt.loc[j,'total_time'] = best_plan[3]
        df_j_opt.loc[j,'scheduled_plan'] = str(best_plan[4])
        df_j_opt.loc[j,'optimal_plan'] = str(best_plan[5])
        df_j_opt.loc[j,'time_schedule'] = str(best_plan[6])        
        
        
        #display(df_j_opt)
        #print(df_j_opt.info())
    best_plan2=df_j_opt.loc[df_j_opt['value'].idxmin(),:]
    #print("returned value is")
    #display(best_plan2)
    return best_plan2

In [ ]:
import os
import pandas as pd

# 最適化の実行
#folder_path = 'july/'  # 入力フォルダ
#folder_path = 'august/'  # 入力フォルダ
folder_path = 'date_data2/'  # 入力フォルダ
#folder_path = 'median_data/'  # 入力フォルダ

output_folder = 'automation_best_result_csv/'  # 出力フォルダ
#output_folder = 'automation_med_best_result/'  # 出力フォルダ

# 出力フォルダが存在しない場合は作成
os.makedirs(output_folder, exist_ok=True)

file_names = sorted(os.listdir(folder_path))  # フォルダ名をソート

for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    if file_path.endswith('.csv'):
        best_csv = []  # 各ファイル用のリストを初期化
        for i in range(100):  # ファイルにつき何回最適化を行うか
            plan = optimization(file_path)
            plan_data = plan#.iloc[1:]
            best_csv.append(plan_data.values.tolist())
            # 追加のロジックをここに記述することもできます

        # best_csvリストをDataFrameに変換
        columns = ["日付","lambda", "トータル満足度", "トータル所要時間", "予定プラン", "最適プラン", "タイムスケジュール"]
        df_best_csv = pd.DataFrame(best_csv, columns=columns)

        # 日付をインデックスに設定
        df_best_csv = df_best_csv.set_index("日付")

        # CSVファイル名を生成
        base_name = os.path.splitext(file_name)[0]  # .csvを除去
        output_file_name = f're_best_results_{base_name}_r100_t300_restrictions_λ_review.csv'
        
        # 出力ファイルのパスを作成
        output_file_path = os.path.join(output_folder, output_file_name)
        
        # DataFrameをCSVファイルとして保存
        df_best_csv.to_csv(output_file_path, index=True)
        print(output_file_name,'　has been output')


re_best_results_2022-07-01_r100_t300_restrictions_λ_review.csv 　has been output
re_best_results_2022-07-02_r100_t300_restrictions_λ_review.csv 　has been output
re_best_results_2022-07-03_r100_t300_restrictions_λ_review.csv 　has been output
re_best_results_2022-07-04_r100_t300_restrictions_λ_review.csv 　has been output
re_best_results_2022-07-05_r100_t300_restrictions_λ_review.csv 　has been output
re_best_results_2022-07-06_r100_t300_restrictions_λ_review.csv 　has been output
re_best_results_2022-07-07_r100_t300_restrictions_λ_review.csv 　has been output
re_best_results_2022-07-08_r100_t300_restrictions_λ_review.csv 　has been output
re_best_results_2022-07-09_r100_t300_restrictions_λ_review.csv 　has been output
re_best_results_2022-07-10_r100_t300_restrictions_λ_review.csv 　has been output
re_best_results_2022-07-11_r100_t300_restrictions_λ_review.csv 　has been output
re_best_results_2022-07-12_r100_t300_restrictions_λ_review.csv 　has been output
re_best_results_2022-07-13_r100_t300_res